used adaboost with grid search, k-folds validation, and feature importance
# ['landon simpson', 'shane wiskus'] - both no good

In [173]:
import pandas as pd
import numpy as np
from scipy import stats
# from mlxtend.preprocessing import minmax_scaling
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.datasets import make_circles, make_classification, make_moons
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, fbeta_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.dummy import DummyClassifier
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, make_scorer, fbeta_score


In [174]:
warnings.filterwarnings("ignore")

In [175]:
df = pd.read_csv('../../Combine_Data/men/ph_encoded.csv')
olymp = pd.read_csv('../../Data/cleandata22-23/encoded_m_olympics_ph.csv')
phnames = pd.read_csv('../../Data/cleandata22-23/encoded_m_olympics_phnames.csv')
pholymp = pd.read_csv('../../Data/cleandata22-23/men22_23.csv')

In [176]:
olymp.duplicated().sum()

27

In [177]:
olymp = olymp.drop_duplicates()

In [178]:
df=df.drop(columns=[ 'round_qual','round_final', 'Nation', 'ND'])


In [179]:
df

,Rank,D,E,Total,year,medal,Name
0,1,6.5,8.766,15.266,2019,1,173
1,2,6.5,8.733,15.233,2019,0,319
2,3,6.5,8.166,14.666,2019,0,303
3,4,6.1,7.600,13.700,2019,0,297
4,5,5.9,7.233,13.133,2019,0,98
...,...,...,...,...,...,...,...
530,24,4.8,6.850,11.650,2020,0,130
531,25,4.9,7.850,12.750,2020,0,203
532,25,4.9,6.600,11.500,2020,0,203
533,26,5.5,5.300,10.800,2020,0,10


In [180]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535 entries, 0 to 534
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rank    535 non-null    int64  
 1   D       535 non-null    float64
 2   E       535 non-null    float64
 3   Total   535 non-null    float64
 4   year    535 non-null    int64  
 5   medal   535 non-null    int64  
 6   Name    535 non-null    int64  
dtypes: float64(3), int64(4)
memory usage: 29.4 KB


# Base line using ZeroR

In [181]:

X = df.drop(columns=['medal'])  
y = df['medal']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=90)

# Initialize  ZeroR classifier
zero_r_clf = DummyClassifier(strategy='most_frequent', random_state=90)

zero_r_clf.fit(X_train, y_train)

# Make predictions on  test data
y_pred = zero_r_clf.predict(X_test)

# Evaluate  model's performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("ZeroR Classifier")
print(f"Accuracy: {accuracy:.4f}")
print(f"Classification Report:\n{report}")
print(f"Confusion Matrix:\n{conf_matrix}")


ZeroR Classifier
Accuracy: 0.9774
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       173
           1       0.00      0.00      0.00         4

    accuracy                           0.98       177
   macro avg       0.49      0.50      0.49       177
weighted avg       0.96      0.98      0.97       177

Confusion Matrix:
[[173   0]
 [  4   0]]


# Model Comparison


In [182]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, fbeta_score

y = df['medal']
X = df.drop(columns=['medal'])


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=90)

# Initialize classifiers
random_forest = RandomForestClassifier(random_state=42)
adaboost = AdaBoostClassifier(random_state=42)
gradient_boosting = GradientBoostingClassifier(random_state=42)
xgboost = XGBClassifier(random_state=42)

# Hard Voting Classifier
voting_classifier_hard = VotingClassifier(estimators=[
    ('random_forest', random_forest),
    ('adaboost', adaboost),
    ('gradient_boosting', gradient_boosting),
    ('xgboost', xgboost)
], voting='hard')

# Soft Voting Classifier
voting_classifier_soft = VotingClassifier(estimators=[
    ('random_forest', random_forest),
    ('adaboost', adaboost),
    ('gradient_boosting', gradient_boosting),
    ('xgboost', xgboost)
], voting='soft')

# Train the classifiers
classifiers = {
    'Random Forest': random_forest,
    'AdaBoost': adaboost,
    'Gradient Boosting': gradient_boosting,
    'XGBoost': xgboost,
    'Voting Classifier (Hard)': voting_classifier_hard,
    'Voting Classifier (Soft)': voting_classifier_soft
}

for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    f2_score = fbeta_score(y_test, y_pred, beta=2)
    report = classification_report(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    print(f"\nClassifier: {name}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F2-Score: {f2_score:.4f}")
    print(f"Classification Report:\n{report}")
    print(f"Confusion Matrix:\n{conf_matrix}")



Classifier: Random Forest
Accuracy: 0.9831
F2-Score: 0.2941
Classification Report:
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       173
           1       1.00      0.25      0.40         4

    accuracy                           0.98       177
   macro avg       0.99      0.62      0.70       177
weighted avg       0.98      0.98      0.98       177

Confusion Matrix:
[[173   0]
 [  3   1]]

Classifier: AdaBoost
Accuracy: 1.0000
F2-Score: 1.0000
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       173
           1       1.00      1.00      1.00         4

    accuracy                           1.00       177
   macro avg       1.00      1.00      1.00       177
weighted avg       1.00      1.00      1.00       177

Confusion Matrix:
[[173   0]
 [  0   4]]

Classifier: Gradient Boosting
Accuracy: 0.9944
F2-Score: 0.9524
Classification Report:
             

In [183]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, VotingClassifier
# from sklearn.tree import DecisionTreeClassifier
# from xgboost import XGBClassifier
# from sklearn.metrics import accuracy_score, fbeta_score, classification_report, confusion_matrix


# X = df.drop(columns=['medal'])
# y = df['medal']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.23, random_state=90)

# # Example AdaBoost Classifier
# ada_clf = AdaBoostClassifier(n_estimators=50, random_state=42)
# ada_clf.fit(X_train, y_train)

# # Example XGBoost Classifier
# xgb_clf = XGBClassifier(n_estimators=50, random_state=42)
# xgb_clf.fit(X_train, y_train)

# # Example Soft Voting Classifier
# soft_voting_clf = VotingClassifier(
#     estimators=[('ada', ada_clf), ('xgb', xgb_clf)],
#     voting='soft'
# )
# soft_voting_clf.fit(X_train, y_train)

# # Evaluate the models
# models = {'AdaBoost': ada_clf, 'XGBoost': xgb_clf, 'Soft Voting': soft_voting_clf}

# for name, model in models.items():
#     y_pred = model.predict(X_test)
    
#     accuracy = accuracy_score(y_test, y_pred)
#     f2_score = fbeta_score(y_test, y_pred, beta=2)
#     report = classification_report(y_test, y_pred)
#     conf_matrix = confusion_matrix(y_test, y_pred)

#     print(f"Classifier: {name}")
#     print(f"Accuracy: {accuracy:.4f}")
#     print(f"F2-Score: {f2_score:.4f}")
#     print(f"Classification Report:\n{report}")
#     print(f"Confusion Matrix:\n{conf_matrix}\n")


In [184]:
# from sklearn.ensemble import VotingClassifier
# from sklearn.base import clone


# # Create a list of base classifiers
# classifiers = [
#     ('AdaBoost', clone(ada_clf)),
#     ('XGBoost', clone(xgb_clf)),
#     ('SoftVoting', clone(soft_voting_clf))
# ]

# # Create the ensemble model (Voting Classifier)
# ensemble_model = VotingClassifier(estimators=classifiers, voting='soft')

# # Train the ensemble model
# ensemble_model.fit(X_train, y_train)

# # Predictions on the test set
# ensemble_predictions = ensemble_model.predict(X_test)

# # Evaluate the ensemble model
# ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
# ensemble_f2_score = fbeta_score(y_test, ensemble_predictions, beta=2)
# ensemble_report = classification_report(y_test, ensemble_predictions)
# ensemble_conf_matrix = confusion_matrix(y_test, ensemble_predictions)

# # Display results for the ensemble model
# print("\nEnsemble Model (AdaBoost, XGBoost, and Soft Voting)")
# print(f"Accuracy: {ensemble_accuracy:.4f}")
# print(f"F2-Score: {ensemble_f2_score:.4f}")
# print(f"Classification Report:\n{ensemble_report}")
# print(f"Confusion Matrix:\n{ensemble_conf_matrix}")


In [185]:
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.base import clone
from sklearn.metrics import accuracy_score, fbeta_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.base import clone

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.23, random_state=90)

# Create base models
ada_clf = AdaBoostClassifier(n_estimators=50, learning_rate=0.1)
xgb_clf = XGBClassifier(n_estimators=50, learning_rate=0.1, max_depth=3)
knn_clf = KNeighborsClassifier(n_neighbors=5)
dt_clf = DecisionTreeClassifier()

# Create the ensemble model (Stacking Classifier)
stacking_model = StackingClassifier(
    estimators=[
        ('AdaBoost', clone(ada_clf)),
        ('XGBoost', clone(xgb_clf)),
        ('KNN', clone(knn_clf)),
        ('DecisionTree', clone(dt_clf))
    ],
    final_estimator=RandomForestClassifier(),  # Meta-model
    stack_method='auto',  # Auto mode for the meta-model fitting strategy
    cv=11  # Number of cross-validation folds for training base models
)

# Train the stacking model
stacking_model.fit(X_train, y_train)

# Predictions on the test set
stacking_predictions = stacking_model.predict(X_test)

# Evaluate the stacking model
stacking_accuracy = accuracy_score(y_test, stacking_predictions)
stacking_f2_score = fbeta_score(y_test, stacking_predictions, beta=2)
stacking_report = classification_report(y_test, stacking_predictions)
stacking_conf_matrix = confusion_matrix(y_test, stacking_predictions)

# Display results for the stacking model
print("\nStacking Model (AdaBoost, XGBoost, KNN, Decision Tree with RandomForest meta-model)")
print(f"Accuracy: {stacking_accuracy:.4f}")
print(f"F2-Score: {stacking_f2_score:.4f}")
print(f"Classification Report:\n{stacking_report}")
print(f"Confusion Matrix:\n{stacking_conf_matrix}")



Stacking Model (AdaBoost, XGBoost, KNN, Decision Tree with RandomForest meta-model)
Accuracy: 1.0000
F2-Score: 1.0000
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       122
           1       1.00      1.00      1.00         2

    accuracy                           1.00       124
   macro avg       1.00      1.00      1.00       124
weighted avg       1.00      1.00      1.00       124

Confusion Matrix:
[[122   0]
 [  0   2]]


In [186]:

y = df['medal']
X = df.drop(columns=['medal'])

# CHANGED RANDOM STATE TO HAVE A LARGER NUMBER OF VALUES IN TEST DF
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.23, random_state=90)

# Initialize classifiers with default hyperparameters
classifiers = {
    "Random Forest": RandomForestClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "SVM": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Naive Bayes": GaussianNB(),
    "Neural Network": MLPClassifier(),
}

# Create a dictionary to store the best hyperparameters for each classifier
best_params = {}

# Iterate through each classifier
for name, clf in classifiers.items():
    # Create a pipeline with StandardScaler for classifiers that require it
    if name in ["SVM", "K-Nearest Neighbors", "Neural Network"]:
        pipeline = make_pipeline(StandardScaler(), clf)
    else:
        pipeline = clf

    # Define parameter grid for hyperparameter tuning
    param_grid = {}  # Add hyperparameters and their possible values to tune

    # Initialize StratifiedKFold for cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Use GridSearchCV for hyperparameter tuning
    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        scoring={'accuracy': 'accuracy', 'f2_score': make_scorer(fbeta_score, beta=2)},
        refit='accuracy',  # Choose the metric to optimize
        cv=cv,
        n_jobs=-1,  # Use all available CPUs
        verbose=1
    )

    # Fit the model to the training data with hyperparameter tuning
    grid_search.fit(X_train, y_train)

    # Get the best hyperparameters and store them
    best_params[name] = grid_search.best_params_

    # Make predictions on the test data using the best model
    y_pred = grid_search.predict(X_test)

    # Evaluate the model's performance
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    # Print the results
    print(f"Classifier: {name}")
    print(f"Best Hyperparameters: {best_params[name]}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F2-Score: {fbeta_score(y_test, y_pred, beta=2):.4f}")
    print(f"Classification Report:\n{report}")
    print(f"Confusion Matrix:\n{conf_matrix}\n")


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Classifier: Random Forest
Best Hyperparameters: {}
Accuracy: 1.0000
F2-Score: 1.0000
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       122
           1       1.00      1.00      1.00         2

    accuracy                           1.00       124
   macro avg       1.00      1.00      1.00       124
weighted avg       1.00      1.00      1.00       124

Confusion Matrix:
[[122   0]
 [  0   2]]

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Classifier: AdaBoost
Best Hyperparameters: {}
Accuracy: 1.0000
F2-Score: 1.0000
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       122
           1       1.00      1.00      1.00         2

    accuracy                           1.00       124
   macro avg       1.00      1.00      1.00       124
weighted avg       1.00      1

# Feature Importance

In [187]:
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import AdaBoostClassifier

# y = df['medal']
# X = df.drop(columns=['medal'])

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.13, random_state=9)

# # Initialize the AdaBoost classifier
# clf = AdaBoostClassifier()

# # Fit the model to the training data
# clf.fit(X_train, y_train)

# # Extract feature importances
# feature_importances = clf.feature_importances_

# # Print feature importances
# print("Feature Importances:")
# for feature, importance in zip(X.columns, feature_importances):
#     print(f"{feature}: {importance:.4f}")


In [188]:
# import matplotlib.pyplot as plt

# feature_importances = clf.feature_importances_

# # Plot feature importances
# plt.figure(figsize=(10, 6))
# plt.barh(X.columns, feature_importances, color='skyblue')
# plt.xlabel('Feature Importance')
# plt.title('AdaBoost Classifier - Feature Importances')
# plt.show()


# # Final Model

In [189]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, make_scorer, fbeta_score

# Set up your X and y
y = df['medal']
X = df.drop(columns=['medal'])

# CHANGED RANDOM STATE TO HAVE A LARGER NUMBER OF VALUES IN TEST DF
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.23, random_state=90)

# Define the AdaBoostClassifier
adaboost_clf = AdaBoostClassifier()

# Define parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [20, 23, 25, 27, 30],
    'learning_rate': [.031, .45, .5, .55],
    # Add other hyperparameters and their possible values here
}

# Initialize StratifiedKFold for cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(
    adaboost_clf,
    param_grid,
    scoring={'accuracy': 'accuracy', 'f2_score': make_scorer(fbeta_score, beta=2)},
    refit= 'f2_score',
    cv=cv,
    n_jobs=-1,  # Use all available CPUs
    verbose=1
)

# Fit the model to the training data with hyperparameter tuning
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Make predictions on the test data using the best model
y_pred = grid_search.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Print the results
print("Best Hyperparameters:", best_params)
print(f"Accuracy: {accuracy:.4f}")
print(f"F2-Score: {fbeta_score(y_test, y_pred, beta=2):.4f}")
print(f"Classification Report:\n{report}")
print(f"Confusion Matrix:\n{conf_matrix}\n")


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Hyperparameters: {'learning_rate': 0.5, 'n_estimators': 20}
Accuracy: 1.0000
F2-Score: 1.0000
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       122
           1       1.00      1.00      1.00         2

    accuracy                           1.00       124
   macro avg       1.00      1.00      1.00       124
weighted avg       1.00      1.00      1.00       124

Confusion Matrix:
[[122   0]
 [  0   2]]



# Model Trained Above, now using 2023/2022 data

In [190]:
olymp = pd.read_csv('../../Data/cleandata22-23/encoded_m_olympics_ph.csv')
olymp = olymp.drop_duplicates()

In [191]:
olymp.rename(columns={'D Score': 'D', 'E Score': 'E',
                     'Pen.': 'ND', 'nation': 'Nation'}, inplace=True)
olymp=olymp.drop(columns=['round_TeamFinal', 'round_AAfinal', 
                          'round_TeamQual', 'ND', 'round_qual','round_final', 'Nation' ])

In [192]:
y_predz = grid_search.predict(olymp)

# y_predz = ensemble_model.predict(olymp)

# y_predz = stacking_model.predict(olymp)

In [193]:
ypreds  = pd.Series(y_predz)
olymp['ypredz']=ypreds

In [194]:
olymp['ypredz'].value_counts()


0.0    2501
1.0       3
Name: ypredz, dtype: int64

In [195]:
filtered_df = olymp[olymp['ypredz'] == 1]

In [196]:
filtered_df['Name'].unique()

array([419, 259])

# phnames used to match encoded names with actual names

In [197]:
# name_encoded_values = [185, 163]

name_encoded_values = [419, 259]
# Filter the DataFrame to get the corresponding "Name" values
result = phnames.loc[phnames['Name_encoded'].isin(name_encoded_values), 'Name']


In [198]:
result.unique()

array(['loran de munck', 'harald wibye'], dtype=object)

# pholymp used to get country info from name list above

In [199]:
pholymp["Name"]=pholymp["Name"].str.lower()

In [200]:
names_to_filter =['loran de munck', 'harald wibye']
# Filter the DataFrame to get rows with the specified "Name" values
result = pholymp[pholymp['Name'].isin(names_to_filter)]

In [201]:
result['Country'].value_counts()

NOR    35
NED    25
Name: Country, dtype: int64

# Find names from USA

In [202]:
usas=result[result['Country']=='USA']

In [203]:
usas['Name'].unique()

array([], dtype=object)